[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/giswqs/gee-tutorials/blob/master/ImageCollection/mapping_over_image_collection.ipynb)

**Mapping over an ImageCollection**

Reference: https://developers.google.com/earth-engine/guides/ic_mapping

To apply a function to every `Image` in an `ImageCollection` use `imageCollection.map()`. The only argument to `map()` is a function which takes one parameter: an `ee.Image`. 

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()

## Define a function

In [ ]:
# Processing is on the server side, so syntax is a bit different than standard Python
def minus1000(num):
    result = ee.Number(num).subtract(1000)
    return result

In [ ]:
result = minus1000(2000)
print(result.getInfo())

## Apply the map function

In [ ]:
years = ee.List.sequence(2000, 2010)
print(years.getInfo())

In [ ]:
# .map acts like a for loop on the server side (maps the function to all items in list)
results = years.map(minus1000)
print(results.getInfo())

## Use anonymous/lambda function

In [ ]:
years = ee.List.sequence(2000, 2010)

results = years.map(lambda x: ee.Number(x).subtract(1000))
print(results.getInfo())

## Filter an ImageCollection

In [ ]:
roi = ee.Geometry.Point(-111.9555, 40.6761)  # TSOB

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate("2020-01-01", "2021-01-01") \
    .sort("CLOUD_COVER")

print(collection.size().getInfo())

In [ ]:
vis_params = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0, 
    'max': 3000,
    'gamma': 1.0
}

image = collection.first()
print(type(image))
Map.addLayer(image, vis_params, 'First image')
Map

In [ ]:
# image.getInfo()

In [ ]:
geemap.image_props(image).getInfo()

In [ ]:
collection.aggregate_array("CLOUD_COVER").getInfo()

In [ ]:
vis_params = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0, 
    'max': 3000,
    'gamma': 1.0
}

Map.addLayer(image, vis_params, "First image")
Map.centerObject(image)
Map

## Select the best August image each year

In [ ]:
# NBR  = (SWIR2 - RED)/(SWIR2 + RED) = (B7 - B4)/(B7 + B4)
def addNBR(x):
  nbr = x.normalizedDifference(['B7', 'B4']).rename('NBR')
  return x.addBands(nbr)

In [ ]:
roi = ee.Geometry.Point(-111.9555, 40.6761)  # TSOB


def best_apr_may_image(year):
    start_date = ee.Date.fromYMD(year, 8, 1)
    end_date = ee.Date.fromYMD(year, 9, 1)
    
    image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .sort("CLOUD_COVER") \
        .first() \
    
    return image

In [ ]:
start_year = 2013
end_year = 2020
years = ee.List.sequence(start_year, end_year)
print(type(years))
year_list = years.getInfo()
print(year_list)

In [ ]:
images = years.map(best_apr_may_image) # images is now a list

In [ ]:
print(type(images.get(0)))

In [ ]:
print(type(images))

In [ ]:
count = images.size().getInfo()
print(count)

In [ ]:
ee.ImageCollection(images).aggregate_array("CLOUD_COVER").getInfo() # convert list to image collection

In [ ]:
for index in range(0, count):
    temp = ee.Image(images.get(index))
#     print(geemap.image_props(temp).getInfo())
    layer_name = "Image " + str(year_list[index])
    Map.addLayer(temp, vis_params, layer_name, False)
    
Map

## Create Time Series with NBRs

In [ ]:
years_str = [str(x) for x in year_list]
print(years_str)
yr_collection = ee.ImageCollection(images).map(addNBR)
years_nbr = yr_collection.select('NBR')

In [ ]:
years_nbr.aggregate_array("CLOUD_COVER").getInfo()
nbrs = years_nbr.toBands()

first = years_nbr.first()
print(geemap.image_props(first).getInfo())

last = years_nbr.sort("system:time_start", False).first()
print(geemap.image_props(last).getInfo())

image_2020 = ee.Image(images.get(7))
print(geemap.image_props(image_2020).getInfo())

In [ ]:
Map = geemap.Map()
nbrParams = {'min': 0, 'max': 0.5, 'palette': ['lightgray','green', 'lightgreen', 'khaki', 'darkkhaki', 'orange', 'darkgoldenrod', 'red', 'darkred']};

Map.addLayer(image_2020, vis_params, '2020', False)
Map.addLayer(nbrs, {}, "NBR time series", False)
Map.add_time_slider(nbrs, nbrParams, labels=years_str, time_interval=1)

Map